In [1]:
import torch
import torch.nn
import sys
import numpy as np
import matplotlib.pyplot as plt
from classifier_1 import ImageClassifier
from dataloader_1 import load_mnist, split_data, get_hidden_sizes

In [2]:
model_fn = "./classifier_1.pth"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
def load(fn, device):
    d = torch.load(fn, map_location=device)
    
    return d['model'], d['config']

In [4]:
def plot(x, y_hat):
    for i in range(x.size(0)):
        img = (np.array(x[i].detach().cpu(), dtype='float')).reshape(28,28)

        plt.imshow(img, cmap='gray')
        plt.show()
        print("Predict:", float(torch.argmax(y_hat[i], dim=-1)))

In [5]:
def test(model, x, y, to_be_shown=True):
    model.eval()
    
    with torch.no_grad():
        # 미니배치 단위로 추론을 수행하도록 개선해야
        y_hat = model(x)

        correct_cnt = (y.squeeze() == torch.argmax(y_hat, dim=-1)).sum()
        total_cnt = float(x.size(0))
        
        accuracy = correct_cnt / total_cnt
        print("Accuracy: %.4f" % accuracy)
        
        if to_be_shown:
            plot(x, y_hat)

In [6]:
model_dict, train_config = load(model_fn, device)

# Load MNIST test set.
X, y = load_mnist(is_train=False)
X, y = X.to(device), y.to(device)

input_size = int(X.shape[-1])
output_size = int(max(y)) + 1

model = ImageClassifier(
    input_size=input_size,
    output_size=output_size,
    hidden_sizes=get_hidden_sizes(input_size,
                                  output_size,
                                  train_config.n_layers),
    use_batch_norm=not train_config.use_dropout,
    dropout_p=train_config.dropout_p,
).to(device)

model.load_state_dict(model_dict)

test(model, X, y, to_be_shown=False)

FileNotFoundError: [Errno 2] No such file or directory: './classifier_1.pth'

In [7]:
n_test = 20
test(model, X[:n_test], y[:n_test], to_be_shown=True)

NameError: name 'model' is not defined